<a href="https://colab.research.google.com/github/AymenTlili131/The-sound-of-Ai-Hack-V1.0/blob/working_proposals/MIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
import librosa
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from os import walk
import librosa.display
from natsort import natsorted
import pickle
from tqdm.notebook import tqdm

In [71]:
from google.colab import drive
###working with a Drive version of the dataset
drive.mount('/content/drive',force_remount= True)

Mounted at /content/drive


In [72]:
def extract_features(x, 
                    #  feature, 
                     win_length,
                     n_mfcc, 
                     sr, 
                     n_mels):
    #MIR extraction 
    hop_length = int(win_length/2)
    rms = librosa.feature.rms(x, frame_length=win_length, hop_length=hop_length, center=True)[0]
    spec_cent = librosa.feature.spectral_centroid(y=x ,sr=sr, hop_length=hop_length, win_length=win_length)[0]  
    zcr = librosa.feature.zero_crossing_rate(x, frame_length=win_length, hop_length=hop_length)[0]
    #D = np.abs(librosa.stft(x))**2
    #mel = librosa.feature.melspectrogram(S=D, sr=sr) 
    mel = librosa.feature.melspectrogram(x, sr= sr, hop_length= hop_length,n_mels= n_mels)
    mfcc = librosa.feature.mfcc(x, sr= sr,n_mfcc= n_mfcc)
    duration = x.shape[0]/sr
    bpm = librosa.beat.tempo(x, sr= sr)[0]

    features = {'rms': rms,
                'mfcc': mfcc,
                'mel': mel,
                'sc': spec_cent,
                'zcr': zcr,
                'duration': duration,
                'bpm': bpm
                }

    return features

In [73]:
artist= os.listdir('/content/drive/MyDrive/Hack v1.0/Dataset/Spotify/')
print(artist)

['Sweden', 'Mexico', 'USA', 'India', 'MIR_Sweden.csv', 'MIR_Mexico.csv', 'MIR_USA.csv', 'MIR_India.csv', 'MIR_summary.csv']


In [74]:
from pathlib import Path

def parse_dataset_wavs(dataset_path: str, verbose: bool=False) -> tuple():
    dirs_wavs = sorted(Path(dataset_path).glob('**/*.wav'))
    # print(len(dirs_wavs))
    artists = [str(x).split('/')[-3] for x in dirs_wavs]
    songs = [str(x).split('/')[-1] for x in dirs_wavs]
    audio_paths = [str(dirs_wavs[idx]) for idx, dir in enumerate(dirs_wavs)]
    if verbose:
        for i in range(len(dirs_wavs)):
            print(f'Artist {i}',artists[i])
            print(f'Song {i}', songs[i])
            print(f'Audio {i}', audio_paths[i])
            print('\n')
    assert len(dirs_wavs) == len(artists) == len(songs) == len(audio_paths)

    return artists, songs, audio_paths

def create_waveforms(audio_path: list, sr: int=44100) -> list:
    audio_files = []
    for audio in audio_path:
        x, fs = librosa.load(audio, sr=sr)
        audio_files.append(x)
    audio_files = np.array(audio_files)

    return audio_files

In [75]:
# import IPython.display as ipd
# ipd.Audio(audio_files[0][:441000],rate=44100)


In [79]:
def create_dataframe(dataset_path, display_dataframe: bool=True, sr=44100, win_length=2048, duration=30.0, n_mfcc=13, n_mels=128):

    artists, songs, audio_paths = parse_dataset_wavs(dataset_path)
    audio_files = create_waveforms(audio_paths, sr)

    MEL, RMS, SC, ZCR, DUR, BPM = [], [], [], [], [], []
    for i in tqdm(audio_files, desc='Extracting features'):
        features_dict = extract_features(i[:int(sr * duration)], win_length=win_length, n_mfcc=n_mfcc, sr=sr, n_mels=n_mels)
        MEL.append(features_dict['mel'])
        RMS.append(np.array(features_dict['rms']))
        SC.append(np.array(features_dict['sc']))
        ZCR.append(np.array(features_dict['zcr']))
        DUR.append(features_dict['duration'])
        BPM.append(features_dict['bpm'])

    dict_info = {
    "artist": artists,
    "song": songs,
    "mel": MEL,
    "rms(mean)": np.mean(RMS, axis=1),
    "rms(var)": np.var(RMS, axis=1),
    "sc(mean)": np.mean(SC, axis=1),
    "sc(var)": np.var(SC, axis=1),
    "zcr(mean)": np.mean(ZCR, axis=1),
    "zcr(var)": np.var(ZCR, axis=1),
    "duration": DUR,
    "bpm": BPM,
    }

    audio_features = pd.DataFrame(dict_info)
    if display_dataframe:
        display(audio_features)
    
    return audio_features

In [80]:
# DATASET_PATH = '/content/drive/MyDrive/Hack v1.0/Dataset/Spotify/'
# dirs = os.listdir(DATASET_PATH)
# print(dirs)

mir_sweden = create_dataframe('/content/drive/MyDrive/Hack v1.0/Dataset/Spotify/Sweden/')
mir_india = create_dataframe('/content/drive/MyDrive/Hack v1.0/Dataset/Spotify/India/')
mir_usa = create_dataframe('/content/drive/MyDrive/Hack v1.0/Dataset/Spotify/USA/')
mir_mexico = create_dataframe('/content/drive/MyDrive/Hack v1.0/Dataset/Spotify/Mexico/')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Extracting features:   0%|          | 0/49 [00:00<?, ?it/s]

,artist,song,mel,rms(mean),rms(var),sc(mean),sc(var),zcr(mean),zcr(var),duration,bpm
0,23,01 Bla bla (Intro).wav,"[[9.290754e-18, 1.9965177e-07, 5.3759522e-05, ...",0.145916,0.008809,2293.129278,1.184585e+06,0.052493,0.001241,30.0,135.999178
1,"Alesso, Zara Larsson",01 Words (feat. Zara Larsson).wav,"[[0.0, 6.4441608e-09, 1.4698058e-05, 1.5179209...",0.159152,0.003780,2620.256999,2.234224e+06,0.058505,0.002622,30.0,126.048018
2,Ant Wan,01 Komplicerat.wav,"[[0.0, 0.0016549706, 0.009627789, 0.015177699,...",0.236703,0.011463,2319.940081,2.584504e+06,0.045590,0.001675,30.0,132.512019
3,BELL,01 MISSFÖRSTÅTT.wav,"[[0.0, 0.0, 0.0, 0.0011059896, 0.12315781, 0.1...",0.235809,0.011439,1961.185749,1.484623e+06,0.039298,0.001049,30.0,109.956782
4,Babblarna,01 Babblarnas vaggvisa.wav,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.077918,0.003925,1233.312531,1.172904e+06,0.024619,0.000440,30.0,92.285156
5,Benson Boone,01 In The Stars.wav,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.097384,0.007683,2327.358398,2.277402e+06,0.050736,0.003789,30.0,143.554688
6,Beyoncé,01 BREAK MY SOUL.wav,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.191381,0.028487,1851.212532,1.755613e+06,0.038733,0.001632,30.0,114.843750
7,Bolaget,01 Kan Inte Gå.wav,"[[1.86534e-06, 7.6159574e-07, 2.9103168e-07, 1...",0.098779,0.001374,2880.289286,2.981776e+06,0.057042,0.004176,30.0,139.674831
8,Cornelia Jakobs,01 Hold Me Closer.wav,"[[0.0, 1.6144596e-15, 1.0312249e-08, 2.7211337...",0.178832,0.007733,2330.989644,1.370606e+06,0.041851,0.001528,30.0,120.185320
9,Dani M,01 Vidare.wav,"[[4.966081e-07, 0.07530498, 0.2711832, 0.00874...",0.106586,0.015931,2110.966626,1.154765e+06,0.038104,0.000784,30.0,97.508844


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Extracting features:   0%|          | 0/49 [00:00<?, ?it/s]

...

Extracting features:   0%|          | 0/49 [00:00<?, ?it/s]

,artist,song,mel,rms(mean),rms(var),sc(mean),sc(var),zcr(mean),zcr(var),duration,bpm
0,Bad Bunny,01 Moscow Mule.wav,"[[0.0008153785, 0.0015700816, 0.011976372, 0.0...",0.156551,0.006353,1906.198156,1.195856e+06,0.031456,0.000453,30.0,99.384014
1,Bad Bunny,02 Después de la Playa.wav,"[[0.0, 5.043622e-08, 7.812954e-06, 0.000162535...",0.119760,0.003906,2460.327444,1.489299e+06,0.054001,0.000844,30.0,147.656250
2,Bad Bunny,04 Tití Me Preguntó.wav,"[[0.0049165906, 0.040400945, 0.029661601, 0.02...",0.036301,0.001849,2029.320892,6.845006e+05,0.036609,0.000781,30.0,123.046875
3,Bad Bunny,10 Efecto.wav,"[[3.1004332e-05, 0.0025607455, 0.0025986864, 0...",0.105675,0.004739,2357.926789,1.492880e+06,0.036793,0.000966,30.0,99.384014
4,"Bad Bunny, Bomba Estéreo",14 Ojitos Lindos.wav,"[[0.00341071, 0.0011381691, 0.0042318804, 0.01...",0.115038,0.001398,1925.010619,1.413014e+06,0.041626,0.000911,30.0,103.359375
5,"Bad Bunny, Chencho Corleone",03 Me Porto Bonito.wav,"[[0.00025073136, 0.006425243, 0.017815989, 0.1...",0.076318,0.002817,2891.532358,1.724735e+06,0.049271,0.001151,30.0,99.384014
6,"Bad Bunny, Rauw Alejandro",11 Party.wav,"[[0.0034770975, 0.0037747633, 0.006301054, 0.0...",0.120820,0.005433,2772.850594,7.485855e+05,0.061691,0.001452,30.0,97.508844
7,Beyoncé,01 BREAK MY SOUL.wav,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.191381,0.028487,1851.212532,1.755613e+06,0.038733,0.001632,30.0,114.843750
8,Brent Faiyaz,01 VILLAIN'S THEME.wav,"[[2.2181966e-06, 8.7401077e-07, 1.9655266e-05,...",0.132248,0.013396,780.610854,4.465540e+05,0.014565,0.000150,30.0,132.512019
9,Brent Faiyaz,02 LOOSE CHANGE.wav,"[[0.0, 8.8685766e-08, 0.00049525197, 0.0111030...",0.148879,0.025078,1162.045710,6.827304e+05,0.023210,0.000430,30.0,166.708669


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Extracting features:   0%|          | 0/50 [00:00<?, ?it/s]

,artist,song,mel,rms(mean),rms(var),sc(mean),sc(var),zcr(mean),zcr(var),duration,bpm
0,Bad Bunny,04 Tití Me Preguntó.wav,"[[0.0049165906, 0.040400945, 0.029661601, 0.02...",0.036301,0.001849,2029.320892,6.845006e+05,0.036609,0.000781,30.0,123.046875
1,Bad Bunny,10 Efecto.wav,"[[3.1004332e-05, 0.0025607455, 0.0025986864, 0...",0.105675,0.004739,2357.926789,1.492880e+06,0.036793,0.000966,30.0,99.384014
2,"Bad Bunny, Bomba Estéreo",14 Ojitos Lindos.wav,"[[0.00341071, 0.0011381691, 0.0042318804, 0.01...",0.115038,0.001398,1925.010619,1.413014e+06,0.041626,0.000911,30.0,103.359375
3,"Bad Bunny, Chencho Corleone",03 Me Porto Bonito.wav,"[[0.00025073136, 0.006425243, 0.017815989, 0.1...",0.076318,0.002817,2891.532358,1.724735e+06,0.049271,0.001151,30.0,99.384014
4,"Banda Elemental de Mazatlán Sinaloa, Julión ...",01 Una Raya Más al Tigre.wav,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.230201,0.013039,3207.890236,1.689471e+06,0.057721,0.001416,30.0,117.453835
5,"Bizarrap, Villano Antillano","01 Villano Antillano_ Bzrp Music Sessions, Vol...","[[0.0, 0.0, 0.0, 0.0, 1.4970375e-08, 159.02943...",0.192772,0.021146,3195.494881,3.752130e+06,0.060308,0.003923,30.0,129.199219
6,Dave Bolaño,01 Me Levanté.wav,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.088562,0.007476,4218.355419,3.502912e+06,0.097145,0.006202,30.0,114.843750
7,El Burger,01 No tienes motivacion.wav,"[[0.0, 0.0, 0.0, 0.0, 0.0, 7.9507534e-10, 2.23...",0.154446,0.013441,3023.562607,1.893879e+06,0.083558,0.004201,30.0,120.185320
8,Feid,01 Ferxxo 100.wav,"[[0.0, 7.501252e-13, 1.5496354, 0.038273163, 0...",0.136335,0.002520,2301.347968,2.564920e+06,0.043241,0.001428,30.0,161.499023
9,Fresto Music,08 Me Hace Daño Verte.wav,"[[0.0, 8.4317e-06, 0.0001668307, 3.6603986e-05...",0.224716,0.010567,3116.047540,1.833542e+06,0.072996,0.002795,30.0,120.185320


In [81]:
DATASET_PATH = '/content/drive/MyDrive/Hack v1.0/Dataset/Spotify/'

mir_india.to_pickle(os.path.join(DATASET_PATH, 'MIR_India.pkl'), compression='bz2', protocol=4)
mir_mexico.to_pickle(os.path.join(DATASET_PATH, 'MIR_Mexico.pkl'), compression='bz2', protocol=4)
mir_sweden.to_pickle(os.path.join(DATASET_PATH, 'MIR_Sweden.pkl'), compression='bz2', protocol=4)
mir_usa.to_pickle(os.path.join(DATASET_PATH, 'MIR_USA.pkl'), compression='bz2', protocol=4)


# mir_sweden.to_csv(os.path.join(DATASET_PATH, 'MIR_Sweden.csv'), encoding='utf-8-sig', index=False)
# mir_mexico.to_csv(os.path.join(DATASET_PATH, 'MIR_Mexico.csv'), encoding='utf-8-sig', index=False)
# mir_usa.to_csv(os.path.join(DATASET_PATH, 'MIR_USA.csv'), encoding='utf-8-sig', index=False)
# mir_india.to_csv(os.path.join(DATASET_PATH, 'MIR_India.csv'), encoding='utf-8-sig', index=False)